In [ ]:
# XGBoost

param_grid={
    'max_depth':[4],
    'learning_rate':[0.1, 0.2, 0.3],
    'gamma':[0],
    'reg_lambda':[1.0],
    'scale_pos_weight': [1]
}

optimal_params = GridSearchCV(
                        estimator=xgb.XGBClassifier(
                            objective='binary:logistic'
                            , tree_method='gpu_hist'
                            , subsample=0.8
                            , colsample_bytree=0.8
                            , seed=42
                            , use_label_encoder=False)
                        ,param_grid=param_grid
                        ,scoring='roc_auc'
                        ,verbose=0
                        ,n_jobs=-1
                        ,cv=3
                        )

optimal_params.fit(X_train,
           y_train,
           verbose=False,
           early_stopping_rounds=10,
           eval_metric='auc',
           eval_set=[(X_test, y_test)])

In [ ]:
# Random Forest

criterion =['gini', 'entropy']
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 5, 10, 15]
# Method of selecting samples for training each tree
bootstrap = [True, False]

param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'criterion': criterion}

optimal_params = RandomizedSearchCV(
                                estimator = RandomForestClassifier()
                                , param_distributions = param_grid
                                , n_iter = 100
                                , cv = 3
                                , verbose=2
                                , random_state=42
                                , n_jobs = -1)

optimal_params.fit(X_processed, y)

In [ ]:
#SVC
from sklearn.model_selection import GridSearchCV

param_grid = [{'kernel' : ('linear', 'poly', 'rbf', 'sigmoid'),
         'C' : [1,5,10],
         'degree' : [3,8],
         'coef0' : [0.01,10,0.5],
         'gamma' : ('auto','scale')
              }]

param_grid = [{
    'C': [0.5, 1, 10, 100],
    'gamma': ['scale', 1, 0.1, 0.01, 0.001, 0.0001],
    'kernel': ['rbf'],
}]

optimal_params = GridSearchCV(
                            SVC(),
                            param_grid,
                            cv=5,
                            scoring='acccuracy'
                            )

optimal_params.fit(X_train, y_train)
print(optimal_params.best_params_)

In [ ]:
#LightGBM using hyperopt

def objective_lgb(space):
    model=lgb.LGBMClassifier(
        n_estimators=int(space['n_estimators']),
        max_depth=int(space['max_depth']),
        num_leaves=int(space['num_leaves']),
        learning_rate=space['learning_rate'],
        gamma=space['gamma'],
        reg_alpha=space['reg_alpha'],
        reg_lambda=space['reg_lambda'],
        colsample_bytree=space['colsample_bytree'],
        min_child_weight=space['min_child_weight'],
        boosting_type=space['boosting_type']
    )
    
    score = -cross_val_score(model, x_train, y_train, cv=10, scoring='roc_auc').mean()
    
    return score
    
    
space= {
    'n_estimators': hp.quniform('n_estimators', 5, 35, 1),
    'max_depth': hp.quniform('max_depth', 5, 15, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(1)),
    'gamma': hp.quniform('gamma', 0.01, 0.1, 0.05),
    'num_leaves': hp.quniform('num_leaves', 5, 50, 1),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
    'min_child_weight': hp.quniform('min_child_weight', 0, 10, 1),
    'boosting_type': hp.choice('boosting_type', ['gbdt', 'dart'])   
}

trials = Trials()

best_param = fmin(fn=objective_lgb, 
                  space=space, 
                  algo=tpe.suggest, 
                  max_evals=100, 
                  trials=trials, 
                  rstate= np.random.RandomState(1)
                 )

print(best_param)

In [ ]:
from scipy.stats import norm, skew 

sns.set_style("whitegrid")
plt.figure(figsize=(12,4))

sns.distplot(housing_df.SalePrice, fit=norm);
(mu, sigma) = norm.fit(housing_df.SalePrice)
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='upper right')

plt.ylabel('Frequency')
plt.title('SalePrice distribution')

# calculate skewness of data
numeric_feats = data.dtypes[data.dtypes != 'object'].index
skewed_feats = data[numeric_feats].apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skewed_feats[abs(skewed_feats) > 0.5]
high_skew

In [ ]:
categorical_features=[]
for c in train_df.columns:
    col_type = train_df[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        train_df[c] = train_df[c].astype('category')
        categorical_features.append(c)
print (categorical_features)

In [ ]:
for cat_feat in categorical_feat:
    data[cat_feat] = data[cat_feat].fillna(str(data[cat_feat][:len(train_df)].value_counts().index[0]))
    
for num_feet in numerical_feat:
    data[num_feet] = data[num_feet].fillna(data[num_feet][:len(train_df)].mean())